In [ ]:
import pandas as pd
df = pd.read_csv("/content/OnlineRetail.csv", sep = "," , encoding= "ISO-8859-1", header = 0)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df_null = round(100*(df.isnull().sum())/len(df),2)

In [ ]:
df_null

In [ ]:
df = df.dropna()

In [ ]:
df.shape

In [ ]:
df['CustomerID'] = df['CustomerID'].astype(str)

In [ ]:
df['Amount'] = df['Quantity']*df['UnitPrice']
rfm = df.groupby('CustomerID')['Amount'].sum()
rfm = rfm.reset_index()
rfm.head()

In [ ]:
rff = df.groupby('CustomerID')['InvoiceNo'].count()
rff = rff.reset_index()
rff.columns = ['CustomerID','Frequency']
rff.head()

In [ ]:
rft = pd.merge(rfm, rff, on = 'CustomerID', how = 'inner')
rft.head()

In [ ]:
  df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format= '%d-%m-%Y %H:%M')

In [ ]:
maxDate = max(df['InvoiceDate'])
maxDate

In [ ]:
df['Diff'] = maxDate - df['InvoiceDate']
df.head()

In [ ]:
rfp = df.groupby('CustomerID')['Diff'].min()
rfp = rfp.reset_index()
rfp.head()

In [ ]:
rfp['Diff'] =rfp['Diff'].dt.days
rfp.head()

In [ ]:
rft = pd.merge(rft, rfp, on = 'CustomerID', how = 'inner')
rft.columns = ['CustomerID', 'Amount', 'Frequency', 'Recency']
rft.head()

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
attributes = ['Amount', 'Frequency', 'Recency']
plt.rcParams['figure.figsize'] = [10,8]
sns.boxplot(data = rft[attributes], orient ="v", palette= "Set2", whis = 1.5, saturation= 1, width  = 0.7)
plt.title("Outliers Detection", fontsize = 15, fontweight = 'bold')
plt.ylabel("Range", fontweight = 'bold')
plt.xlabel("Attributes", fontweight= 'bold')

In [ ]:
Q1 = rft.Amount.quantile(0.05)
Q3 = rft.Amount.quantile(0.95)
IQR = Q3 - Q1
rft = rft[(rft.Amount >= Q1 - 1.5*IQR) & (rfm.Amount <= Q3 + 1.5*IQR)]

Q1 = rft.Recency.quantile(0.05)
Q3 = rft.Recency.quantile(0.95)
IQR = Q3 - Q1
rft = rft[(rft.Recency >= Q1 - 1.5*IQR) & (rft.Recency <= Q3 + 1.5 * IQR)]


Q1 = rft.Frequency.quantile(0.05)
Q3 = rft.Frequency.quantile(0.95)
IQR = Q3 - Q1
rft = rft[(rft.Frequency >= Q1 - 1.5*IQR) & (rft.Frequency <= Q3 + 1.5 * IQR)]

In [ ]:
rft_df = rft[['Amount', 'Frequency', 'Recency']]
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
rft_df_scaled = scaler.fit_transform(rft_df)
rft_df_scaled.shape

In [ ]:
rft_df_scaled = pd.DataFrame(rft_df_scaled)
rft_df_scaled.columns = ['Amount', 'Frequency', 'Recency']
rft_df_scaled.head()

In [ ]:
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
merg = linkage(rft_df_scaled, method = "single", metric = "euclidean")
dendrogram(merg)
plt.show()

In [ ]:
merg = linkage(rft_df_scaled, method = "complete", metric = "euclidean")
dendrogram(merg)
plt.show()

In [ ]:
from scipy.cluster.hierarchy import cut_tree
clus_lab = cut_tree(merg, n_clusters = 2).reshape(-1,)
clus_lab

In [ ]:
rft['Cluster_Labels'] = clus_lab
rft.head()